# Cross-Validation and the Bootstrap



In this lab, we explore the resampling techniques covered in this
chapter. Some of the commands in this lab may take a while to run on
your computer.

We again begin by placing most of our imports at this top level.

In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from sklearn.model_selection import train_test_split


There are several new imports needed for this lab.

In [2]:
from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm


We use the validation set approach to estimate test error rates for different linear models on the Auto dataset. The data (392 observations) are split into equal training and validation sets of 196 each using train_test_split() with test_size=196. Setting a random seed (random_state=0) ensures that the split—and thus the results—can be reproduced exactly.

In [4]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto,
                                         test_size=196,
                                         random_state=0)


Now we can fit a linear regression using only the observations corresponding to the training set `Auto_train`.

In [5]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()


We now use the `predict()` method of `results` evaluated on the model matrix for this model
created using the validation data set. We also calculate the validation MSE of our model.

In [6]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)


np.float64(23.61661706966988)

Hence our estimate for the validation MSE of  the linear regression
fit is $23.62$.

We can also estimate the validation error for
higher-degree polynomial regressions. We first provide a function `evalMSE()` that takes a model string as well
as training and test sets and returns the MSE on the test set.

In [8]:
def evalMSE(terms,
            response,
            train,
            test):

   mm = MS(terms)
   X_train = mm.fit_transform(train)
   y_train = train[response]

   X_test = mm.transform(test)
   y_test = test[response]

   results = sm.OLS(y_train, X_train).fit()
   test_pred = results.predict(X_test)

   return np.mean((y_test - test_pred)**2)


Let’s use this function to estimate the validation MSE
using linear, quadratic and cubic fits. We use the `enumerate()`  function
here, which gives both the values and indices of objects as one iterates
over a for loop.

In [9]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid)
MSE


array([23.61661707, 18.76303135, 18.79694163])

These error rates are $23.62, 18.76$, and $18.80$, respectively. If we
choose a different training/validation split instead, then we
can expect somewhat different errors on the validation set.

In [10]:
Auto_train, Auto_valid = train_test_split(Auto,
                                          test_size=196,
                                          random_state=3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

Using the training/validation split, the validation set errors for models with linear, quadratic, and cubic terms in horsepower are 20.76, 16.95, and 16.97, respectively. These results confirm that a quadratic model predicts mpg better than a linear model, while adding a cubic term provides no meaningful improvement.

## Cross-Validation
In theory, cross-validation can be applied to any generalized linear model (GLM). In practice, however, Python’s simplest tools for cross-validation are in sklearn, which uses a different API than statsmodels. To bridge this gap, the ISLP package provides a wrapper, sklearn_sm(), that allows models fit with statsmodels to be used with sklearn’s cross-validation functions.

The sklearn_sm() class takes a statsmodels model as its first argument, with optional model_str for specifying a formula and model_args for additional fitting parameters (e.g., model_args={'family': sm.families.Binomial()} for logistic regression).

In [12]:
hp_model = sklearn_sm(sm.OLS,
                      MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model,
                            X,
                            Y,
                            cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err


np.float64(24.231513517929212)

The cross_validate() function takes a model with fit(), predict(), and score() methods, along with feature matrix X and response Y. By specifying cv as an integer, we perform $k$-fold cross-validation; using the total number of observations gives leave-one-out CV (LOOCV). Here, the function returns several outputs, and the cross-validated test MSE is estimated to be 24.23.

We can extend this approach to polynomial regressions of higher degrees. Using a for loop, we fit polynomials of degree 1 to 5, compute their cross-validation errors, and store the results in the cv_error vector, where d represents the polynomial degree. The vector is initialized beforehand, and running the loop may take a few seconds.

In [13]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error


array([24.23151352, 19.24821312, 19.33498406, 19.42443031, 19.03320428])

Figure 5.4 shows a steep drop in test MSE from the linear to quadratic fit, with little improvement for higher-degree polynomials.

The outer() method of np.power() (and similar functions like add() or min()) takes two arrays and applies the operation to every pair of elements, producing a larger array.

If you want, I can make it even snappier in one sentence. Do you want me to do that?

In [14]:
A = np.array([3, 5, 9])
B = np.array([2, 4])
np.add.outer(A, B)


array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

In the CV example above, we used $k=n$, but of course we can also use $k<n$. The code is very similar
to the above (and is significantly faster). Here we use `KFold()` to partition the data into $k=10$ random groups. We use `random_state` to set a random seed and initialize a vector `cv_error` in which we will store the CV errors corresponding to the
polynomial fits of degrees one to five.

In [15]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10,
           shuffle=True,
           random_state=0) # use same splits for each degree
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error


array([24.20766449, 19.18533142, 19.27626666, 19.47848404, 19.13722016])

The computation is much faster than LOOCV. While LOOCV could theoretically be quicker for linear models using formula (5.2), cross_validate() doesn’t utilize it. Again, cubic or higher-degree polynomials offer little improvement over a quadratic fit.

The `cross_validate()` function is flexible and can take
different splitting mechanisms as an argument. For instance, one can use the `ShuffleSplit()`
function to implement
the validation set approach just as easily as $k$-fold cross-validation.

In [16]:
validation = ShuffleSplit(n_splits=1,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation);
results['test_score']


array([23.61661707])

One can estimate the variability in the test error by running the following:

In [17]:
validation = ShuffleSplit(n_splits=10,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation)
results['test_score'].mean(), results['test_score'].std()


(np.float64(23.802232661034164), np.float64(1.4218450941091847))

The standard deviation shown doesn’t reflect the true sampling variability of the mean or individual test scores, since overlapping training samples create correlations, but it does indicate the Monte Carlo variation from different random folds.

## The Bootstrap

The bootstrap can be applied in nearly any situation without complex math. We illustrate it using the Section 5.2 example and the Auto dataset to estimate linear regression accuracy.

To estimate a statistic’s accuracy, we can implement a simple bootstrap function in Python. For example, using the Portfolio dataset from ISLP, we estimate the variance of parameter $\alpha$ (formula 5.7) with a function alpha_func() that takes a dataframe D and an index vector idx, returning the estimate of $\alpha$ for the selected observations.

In [18]:
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
   cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
   return ((cov_[1,1] - cov_[0,1]) /
           (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))


This function returns an estimate for $\alpha$
based on applying the minimum
    variance formula (5.7) to the observations indexed by
the argument `idx`.  For instance, the following command
estimates $\alpha$ using all 100 observations.

In [19]:
alpha_func(Portfolio, range(100))

np.float64(0.57583207459283)

Next we randomly select
100 observations from `range(100)`, with replacement. This is equivalent
to constructing a new bootstrap data set and recomputing $\hat{\alpha}$
based on the new data set.

In [20]:
rng = np.random.default_rng(0)
alpha_func(Portfolio,
           rng.choice(100,
                      100,
                      replace=True))

np.float64(0.6074452469619004)

This process can be generalized to create a simple function `boot_SE()` for
computing the bootstrap standard error for arbitrary
functions that take only a data frame as an argument.

In [21]:
def boot_SE(func,
            D,
            n=None,
            B=1000,
            seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,
                         n,
                         replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

Notice the use of `_` as a loop variable in `for _ in range(B)`. This is often used if the value of the counter is
unimportant and simply makes sure  the loop is executed `B` times.

Let’s use our function to evaluate the accuracy of our
estimate of $\alpha$ using $B=1{,}000$ bootstrap replications. 

In [22]:
alpha_SE = boot_SE(alpha_func,
                   Portfolio,
                   B=1000,
                   seed=0)
alpha_SE


np.float64(0.09118176521277699)

The bootstrap estimate of ${\rm SE}(\hat{\alpha})$ is 0.0912.

Estimating the Accuracy of a Linear Regression Model

The bootstrap can assess the variability of regression coefficients and predictions. For the Auto dataset, we use it to evaluate the intercept $\beta_0$ and slope $\beta_1$ of the model predicting mpg from horsepower, comparing the results to the standard SE formulas from Section 3.1.2.

To apply boot_SE(), we write a function taking a dataframe D and indices idx. For bootstrapping a specific regression model, we create a generic boot_OLS() function that uses a formula. The clone() function copies the formula so it can be refit on resampled data, ensuring features like poly() are correctly recomputed.

In [23]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

This is not quite what is needed as the first argument to
`boot_SE()`. The first two arguments which specify the model will not change in the
bootstrap process, and we would like to *freeze* them.   The
function `partial()` from the `functools` module  does precisely this: it takes a function
as an argument, and freezes some of its arguments, starting from the
left. We use it to freeze the first two model-formula arguments of `boot_OLS()`.

In [24]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')


Typing `hp_func?` will show that it has two arguments `D`
and `idx` --- it is a version of `boot_OLS()` with the first
two arguments frozen --- and hence is ideal as the first argument for `boot_SE()`.

The `hp_func()` function can now be used in order to create
bootstrap estimates for the intercept and slope terms by randomly
sampling from among the observations with replacement. We first
demonstrate its utility on 10 bootstrap samples.

In [25]:
rng = np.random.default_rng(0)
np.array([hp_func(Auto,
          rng.choice(Auto.index,
                     392,
                     replace=True)) for _ in range(10)])


array([[39.12226577, -0.1555926 ],
       [37.18648613, -0.13915813],
       [37.46989244, -0.14112749],
       [38.56723252, -0.14830116],
       [38.95495707, -0.15315141],
       [39.12563927, -0.15261044],
       [38.45763251, -0.14767251],
       [38.43372587, -0.15019447],
       [37.87581142, -0.1409544 ],
       [37.95949036, -0.1451333 ]])

Next, we use the `boot_SE()` {}  function to compute the standard
errors of 1,000 bootstrap estimates for the intercept and slope terms.

In [26]:
hp_se = boot_SE(hp_func,
                Auto,
                B=1000,
                seed=10)
hp_se


intercept     0.731176
horsepower    0.006092
dtype: float64

This indicates that the bootstrap estimate for ${\rm SE}(\hat{\beta}_0)$ is
0.85, and that the bootstrap
estimate for ${\rm SE}(\hat{\beta}_1)$ is
0.0074.  As discussed in
Section 3.1.2, standard formulas can be used to compute
the standard errors for the regression coefficients in a linear
model. These can be obtained using the `summarize()`  function
from `ISLP.sm`.

In [27]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se


intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

The standard errors for $\hat{\beta}_0$ and $\hat{\beta}_1$ from the formulas in Section 3.1.2 are 0.717 and 0.006, respectively, which differ from the bootstrap estimates. This doesn’t indicate a problem with the bootstrap; rather, it reflects that the standard formulas rely on assumptions—like estimating $\sigma^2$ from the residuals and treating $x_i$ as fixed—that may not hold. Because the data show a non-linear pattern, the linear model inflates residuals and $\hat{\sigma}^2$, while the bootstrap avoids these assumptions, likely giving more accurate SE estimates.

Fitting a quadratic model improves the fit, and the bootstrap and standard SE estimates now align more closely for $\hat{\beta}_0$, $\hat{\beta}_1$, and $\hat{\beta}_2$.

In [28]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS,
                    quad_model,
                    'mpg')
boot_SE(quad_func, Auto, B=1000)


intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64

We  compare the results to the standard errors computed using `sm.OLS()`.

In [29]:
M = sm.OLS(Auto['mpg'],
           quad_model.fit_transform(Auto))
summarize(M.fit())['std err']


intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64